In [1]:
import time
start_time = time.time()
import numpy as np
import blowfish
import rsa
from Cryptodome.Cipher import AES
from Cryptodome.Cipher import DES
from PIL import Image
import cv2
import math

%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
image = cv2.imread('bird.jpg', 1)
cv2.imshow('original', image)

In [3]:
img_str = cv2.imencode('.jpg', image)[1].tobytes()

In [4]:
if len(img_str)%8 != 0:#check for compatibility
    while len(img_str)%8 != 0:
        img_str += b'\x00'

og_img_np = np.frombuffer(img_str, dtype=np.uint8)

In [5]:
key_d = b'despasci'
cipher_d = DES.new(key_d, DES.MODE_EAX)
nonce_d = cipher_d.nonce
msg_des = cipher_d.encrypt(img_str)

In [6]:
key_d1 = b'despasci'
cipher_d1 = DES.new(key_d, DES.MODE_EAX)
nonce_d1= cipher_d.nonce
msg_des1 = cipher_d.encrypt(msg_des)

In [7]:
cipher_d1 = DES.new(key_d1, DES.MODE_EAX, nonce_d1)
msg_des_dec1 = cipher_d1.decrypt(msg_des1)

In [8]:
cipher_d = DES.new(key_d, DES.MODE_EAX, nonce_d)
msg_des_dec = cipher_d.decrypt(msg_des_dec1)


In [9]:
img_str_np_enc_des = np.frombuffer(msg_des_dec, dtype=np.uint8)
a = len(img_str_np_enc_des)
i = 1
while a%2 == 0:
    i = i*2
    a = a//2

img_str_np_enc_des1 = img_str_np_enc_des.reshape(i,len(img_str_np_enc_des)//i)

In [10]:
enc_img_des = Image.fromarray(img_str_np_enc_des1, 'L')
enc_img_des.show(title='des encrypted')

In [11]:
img_str_np = np.frombuffer(img_str, dtype=np.uint8)
img_str_np

array([255, 216, 255, ...,   0,   0,   0], dtype=uint8)

In [12]:
img = cv2.imdecode(img_str_np, 1)

In [13]:
cv2.imshow('decrypted', img)

In [14]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

d = psnr(image, img)
print(d)

43.22586472210333


In [15]:
cv2.PSNR(image, img)

42.722213649793986

In [16]:
Y = np.square(np.subtract(image,img)).mean()
print("MSE:", Y)

MSE: 3.0938117793787896


In [17]:
np.corrcoef(image.ravel(), img.ravel())

array([[1.        , 0.99943421],
       [0.99943421, 1.        ]])

In [18]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 5.369300842285156 seconds ---
